In [2]:
import sys
python_version = sys.version_info[0]

In [3]:
# If you're running on Colab, you'll need to install the What-if Tool package and authenticate on the TF instance
def pip_install(module):
    if python_version == '2':
        !pip install {module} --quiet
    else:
        !pip3 install {module} --quiet

try:
    import google.colab
    IN_COLAB = True
except:
    IN_COLAB = False

if IN_COLAB:
    pip_install('witwidget')

    from google.colab import auth
    auth.authenticate_user()

In [4]:
pip install witwidget

Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: pyparsing!=3.0.0,!=3.0.1,!=3.0.2,!=3.0.3,<4,>=2.4.2 in c:\users\equipo\anaconda3\lib\site-packages (from httplib2<1dev,>=0.15.0->google-api-python-client>=1.7.8->witwidget) (3.0.4)



In [5]:
!jupyter nbextension install --py --symlink --sys-prefix witwidget

2022-07-21 14:09:40.179971: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cudart64_110.dll'; dlerror: cudart64_110.dll not found
2022-07-21 14:09:40.180035: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
Installing C:\Users\Equipo\anaconda3\lib\site-packages\witwidget\static -> wit-widget
- Validating: ok

    To initialize this nbextension in the browser every time the notebook (or other app) loads:
    
          jupyter nbextension enable witwidget --py --sys-prefix
    


In [6]:
!jupyter nbextension enable --py --sys-prefix witwidget

2022-07-21 14:09:45.620530: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cudart64_110.dll'; dlerror: cudart64_110.dll not found
2022-07-21 14:09:45.620571: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
Enabling notebook extension wit-widget/extension...
      - Validating: ok


In [7]:
import pandas as pd
import numpy as np
import witwidget

from witwidget.notebook.visualization import WitWidget, WitConfigBuilder

In [8]:
import dalex as dx
import pandas as pd
data = pd.read_csv ('synthetic_credit_card_approval.csv')
data

,Num_Children,Group,Income,Own_Car,Own_Housing,Target
0,4,0,36151,0,0,0
1,0,0,36095,1,0,0
2,2,1,62110,1,1,0
3,0,1,73644,1,0,0
4,3,0,99146,0,0,1
...,...,...,...,...,...,...
499995,1,0,43299,1,1,0
499996,8,1,75956,1,1,1
499997,3,0,66476,0,0,0
499998,2,1,74524,0,0,0


In [9]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
X = data.drop(["Target"],axis=1) # axis: {0 or ‘index’, 1 or ‘columns’}, default 0
y = data["Target"]

X_train, X_test, y_train, y_test = train_test_split(X,y,train_size=0.8, test_size=0.2, random_state=0)
print("Data sucessfully loaded!")

Data sucessfully loaded!


In [10]:
X_test = X_test.to_numpy()

In [11]:
y_test = y_test.to_numpy()

In [12]:
X_test

array([[    1,     1, 92093,     1,     1],
       [    2,     0, 81071,     1,     0],
       [    4,     0, 83536,     0,     0],
       ...,
       [    2,     0, 67481,     1,     0],
       [    4,     1, 68047,     1,     0],
       [    1,     1, 83803,     1,     1]], dtype=int64)

In [13]:
# Combine the features and labels into one array for the What if Tool
test_examples = np.hstack((X_test,y_test.reshape(-1,1)))

In [14]:
import xgboost
model = xgboost.XGBClassifier().fit(X_train, y_train)
y_test_predict = model.predict(X_test)
y_test_predict

array([1, 0, 0, ..., 0, 0, 1])

In [15]:
# Create a What-if Tool visualization, it may take a minute to load
# See the cell below this for exploration ideas

# This prediction adjustment function is needed as this xgboost model's
# prediction returns just a score for the positive class of the binary
# classification, whereas the What-If Tool expects a list of scores for each
# class (in this case, both the negative class and the positive class).

def adjust_prediction(y_test_predict):
  return [1 - y_test_predict, y_test_predict]

config_builder = (WitConfigBuilder(test_examples.tolist(), data.columns.tolist() + ['Target'])
  .set_ai_platform_model('fairdetect', 'testset1', 'v1', adjust_prediction=adjust_prediction)
  .set_target_feature('Target')
  .set_label_vocab([0, 1]))
WitWidget(config_builder, height=1000)

WitWidget(config={'model_type': 'classification', 'label_vocab': [0, 1], 'feature_names': ['Num_Children', 'Gr…

In [16]:
config_builder